### 이번 과제는 Bert Model을 사용하여 BBC 뉴스 기사의 category를 분류해보는 과제입니다. clone coding을 하시되, 코드 주석을 line by line으로 꼼꼼하게 달아보시며 공부해보세요!

로컬에서 돌리셔도 되지만, colab에서 GPU로 돌려보는 것을 권장합니다!

## 데이터 로드 및 탐색

In [40]:
%%capture
!pip install transformers

In [41]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KUBIG-NLP/4주차/bbc-text.csv') # bbc-text.csv 파일 경로

In [ ]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [ ]:
print(len(df))

2225


In [ ]:
df.groupby('category').count()

,text
category,
business,510
entertainment,386
politics,417
sport,511
tech,401


## BertTokenizer

토크나이저로 pretrain된 BERT의 BertTokenizer를 갖고 옵니다. 여러 종류를 시도해보세요.

- bert-base-uncased : 108MB param, all lowercase
- bert-large-cased : 340MB param, both upper and lower
- bert-base-cased : 108MB param, multi language, both upper and lower


In [44]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          }

## Dataset

In [45]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        # input_ids를 뽑아내는 부분 => 문자열로 된 라벨을 정수로 바꿈
        self.labels = [labels[label] for label in df['category']]
        # masking을 하는 부분
        # 문장을 토큰화한 후 최대길이로 패딩, truncation을 통해 512 초과하는 문장 자름
        # keys(): input_ids, toekn_type_ids, attention_mask(실제 단어 1, 패딩 0)가 나오게 됨!
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

## Train & Evaluate BertClassifier

pretrain된 BertModel을 불러옵니다. 다른 간단한 층들도 같이 쌓아줍니다.

- bert-base-cased: 12-layer, 768-hidden, 12-self attention heads, 110M parameters. Trained on cased English text.


다른 종류들의 pretrianed model은 아래 링크에서 확인할 수 있습니다.

https://huggingface.co/transformers/v2.9.1/pretrained_models.html

**Bert의 특징**: token embedding, sgement embedding, position embedding을 사용한다!
- token embedding: word piece 임베딩 방식을 사용하며, 자주 나오는 단어는 그대로, 자주 나오지 않는 단어는 더 쪼개 embedding을 사용 + 문장의 끝에 `SEP` 토큰을 사용
- segment embeddingL 토큰으로 나누어진 단어들을 다시 하나의 문장으로 만들고 masking을 통해 각 문장을 구분
- position encoding: transformer의 self attention 모델을 사용 => 입력 순서가 보존되지 않음

**추가적인 특징**

- MLM(Masked Language Model): 일련의 단어가 주어지면 그 단어를 예측 => 문맥 파악(15% 토큰 mask로 바꿈)
- NSP (Next Sentence Predicition): 두 문장의 관계를 이

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5) # 768 차원을 5개 class로 줄
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        # input_ids: 입력 토큰의 ID 텐서
        # attention_maks: 패딩된 부분을 무시
        # pooled_output: CLS 토큰(문장을 대표하는 의미로 사용)의 최종 벡터 표현
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output) # 과적합 방지
        linear_output = self.linear(dropout_output)
        # relu를 사용할 경우 일부 출력값이 0이 될 수 있어 softmax 후 확률이 이상하게 분포될 수 있음
        final_layer = self.relu(linear_output)

        # crossentropy loss를 사용하기 때문에 선형변환 결과를 그대로 반환해야하므로 relu를 제거하는 것도 하나의 방법이 될 수 있
        return final_layer

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0 # 학습 정학도
            total_loss_train = 0 # 학습 손실


            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device) # attention mast 이동
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')


In [46]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1780 222 223


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
EPOCHS = 2 #EPOCH 수 늘려보기!
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

100%|██████████| 890/890 [03:05<00:00,  4.80it/s]


Epochs: 1 | Train Loss:  0.715 | Train Accuracy:  0.422 | Val Loss:  0.561 | Val Accuracy:  0.685


100%|██████████| 890/890 [03:10<00:00,  4.66it/s]


Epochs: 2 | Train Loss:  0.412 | Train Accuracy:  0.815 | Val Loss:  0.226 | Val Accuracy:  0.973


In [ ]:
evaluate(model, df_test)

Test Accuracy:  0.982


# optional) 다양한 시도를 해보셨다면 시도 별 간단한 해석도 달아주세요! 🤗

더 강력한 모델 사용 + 옵티마이저 변경(RAdam)

bert-large-case 모델 사용!
- 예측: 오히려 성능이 낮아질 듯, 그렇게 대규모 데이터셋이 아님!

In [48]:
tokenizer = BertTokenizer.from_pretrained('bert-large-cased')

모델의 구조 바꾸기

1. 모델을 bert-large-cased로 바꿔봄!



In [47]:
class BertClassifierModify(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifierModify, self).__init__()

        self.bert = BertModel.from_pretrained('bert-large-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(1024, 5) # 1024 차원을 5개 class로 줄
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output) # 과적합 방지
        linear_output = self.linear(dropout_output)
        # relu를 사용할 경우 일부 출력값이 0이 될 수 있어 softmax 후 확률이 이상하게 분포될 수 있음
        final_layer = self.relu(linear_output)

        # crossentropy loss를 사용하기 때문에 선형변환 결과를 그대로 반환해야하므로 relu를 제거하는 것도 하나의 방법이 될 수 있
        return final_layer

**optimizer을 바꿔보기!**

▶ recified Adam을 사용하여 학습 초반의 불안정성 문제를 해결(어차피 훈련을 오래하기에 컴퓨팅 자원이 딸리니까 초반에 빠르게 최적화가 가능하도록 해보자!)
- Adam: 학습 초반 학습률이 지나치게 커질 수 있dma
- RAdam: 학습 초반의 학습률을 자동으로 낮아지도록 보정한다 => 데이터셋 크기가 작은 경우 더 일관된 성능 보임 (**데이터의 개수가 2225개이므로 소규모 데이터셋이라고 볼 수 있음**)

In [56]:
from torch.optim import RAdam

def train_modify(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    #print(use_cuda)
    # 자꾸 use_cuda가 1이라고 나오는데 실제 사용은 못함
    use_cuda=False
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = RAdam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0 # 학습 정학도
            total_loss_train = 0 # 학습 손실


            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device) # attention mast 이동
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')


In [50]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1780 222 223


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [57]:
EPOCHS = 2 #EPOCH 수 늘려보기!
model = BertClassifierModify() # 대규모 VS 소규모
LR = 1e-6

train_modify(model, df_train, df_val, LR, EPOCHS)

  1%|▏         | 12/890 [06:59<8:32:05, 34.99s/it]


KeyboardInterrupt: 

In [35]:
evaluate(model, df_test)

Test Accuracy:  0.991


⭐ 큰 모델로 돌렸을 떄 정확도 0.991이 나옴 => 더 나은 성능을 보인다! 근데 epoch 2개 돌리는데 30분이나 걸린다.. 예상과 다르게 큰 모델에서 성능이 더 좋게 나왔다..!
- 자꾸 gpu를 다 씀.. cpu로 돌리면 8시간걸림.. optimizer를 다른 것으로 돌리는건 시간되면 해봐야할 듯!